# Baseline Models
Basic baseline models for next utterance classificationusing completely random predictor, TF-IDF, word2vec (with averaging) and doc2vec methods for vectorizing messages. Evaluation using recall@k on test set with cosine similarity.

In [2]:
import urllib
import pickle
import numpy as np
import pandas as pd
from copy import copy
from gensim.models import doc2vec, word2vec
import gensim
from collections import namedtuple
import timeit
import csv

from modules.evaluation_metrics import recall_at_k
import modules.tfidf
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Read in data
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
valid = pd.read_csv("data/valid.csv")

In [7]:
print train.shape
train.head()

(1000000, 3)


,Context,Utterance,Label
0,i think we could import the old comment via rs...,basic each xfree86 upload will not forc user t...,1
1,i 'm not suggest all - onli the one you modifi...,sorri __eou__ i think it be ubuntu relat . __e...,0
2,afternoon all __eou__ not entir relat to warti...,"yep . __eou__ oh , okay . i wonder what happen...",0
3,interest __eou__ grub-instal work with / be ex...,that the one __eou__,1
4,and becaus python give mark a woodi __eou__ __...,( i think someon be go to make a joke about .a...,1


In [8]:
print test.shape
test.head()

(18920, 11)


,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,anyon know whi my stock oneir export env var u...,nice thank ! __eou__,"wrong channel for it , but check efnet.org , u...","everi time the kernel chang , you will lose vi...",ok __eou__,! nomodeset > acer __eou__ i 'm assum it be a ...,http : //www.ubuntu.com/project/about-ubuntu/d...,thx __eou__ unfortun the program be n't instal...,how can i check ? by do a recoveri for test ? ...,my humbl apolog __eou__,# ubuntu-offtop __eou__
1,i set up my hd such that i have to type a pass...,"so you dont know , ok , anyon els ? __eou__ yo...","nmap be nice , but it be n't what i be look fo...",ok __eou__,cdrom work fine on window . __eou__ i dont thi...,"ah yes , i have read return as rerun __eou__",hm ? __eou__,"not the case , lts be everi other .04 releas ....",pretti much __eou__,i use the one i download from amd __eou__,"ffmpeg be part of the packag , quixotedon , at..."
2,im tri to use ubuntu on my macbook pro retina ...,just wonder how it run __eou__,"yes , that 's what i do , export it to a `` id...",noth - i be talk about the question of myhero ...,that should fix the font be too larg __eou__,"okay , so hcitool echo back hci0 < mac address...",i get to the menu with option such as tri ubun...,whi do u need analyz __eou__ it be a toy __eou...,cntrl-c may stop the command but it doe n't fi...,"if you re onli go to run ubuntu , just get a n...",the one which be not pick up at the moment be ...
3,no suggest ? __eou__ link ? __eou__ how can i ...,you cant load anyth via usb or cd when luk be ...,-p sorri ... __eou__ nmap -p22 __eou__ it doe ...,i guess so i ca n't even launch it . __eou__,note __eou__,rxvt-unicod be one __eou__,i tar all of ~ __eou__,i tar all of ~ __eou__,"i do n't realli know if i can help , but i be ...","that work just fine , thank ! __eou__",thank you __eou__
4,i just ad a second usb printer but not sure wh...,i be set it up under the printer configur __eo...,i 'd say the most common venu would be via lau...,"the old hardi man page , http : //manpages.ubu...",i ll give a tri __eou__,"by the way , the url you post for davf be from...",http : //ubuntuforums.org/showthread.php ? t=1...,"so i load up putti gui , then what do i do ? _...","you should read error messag , it say be you r...",wait the colleg semest to close just to make s...,i be call myself a jerk . all i know be that y...


In [15]:
print test.iloc[0]

Context                   anyon know whi my stock oneir export env var u...
Ground Truth Utterance                                 nice thank ! __eou__
Distractor_0              wrong channel for it , but check efnet.org , u...
Distractor_1              everi time the kernel chang , you will lose vi...
Distractor_2                                                     ok __eou__
Distractor_3              ! nomodeset > acer __eou__ i 'm assum it be a ...
Distractor_4              http : //www.ubuntu.com/project/about-ubuntu/d...
Distractor_5              thx __eou__ unfortun the program be n't instal...
Distractor_6              how can i check ? by do a recoveri for test ? ...
Distractor_7                                        my humbl apolog __eou__
Distractor_8                                        # ubuntu-offtop __eou__
Name: 0, dtype: object


In [10]:
print valid.shape
valid.head()

(19560, 11)


,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,ani idea on how lts will be releas ? __eou__ _...,we be talk 12.04 not 10.04 __eou__,you rememb my flash issu from yesterday or the...,"oh , no idea other be probabl ok __eou__ updat...","no , greenit be say his download speed be slow...",lsb_releas -sc __eou__ well ... regardless . i...,you can buy _anything_ in china __eou__,no __eou__,sudo restart lightdm __eou__,you be still ask for the uniti logout menu rig...,"so i be work as a linux admin intern , and my ..."
1,how much hdd use ubuntu default instal ? __eou...,that whi i ask how much be default instal ? : ...,all of this possibl in older version of ubuntu...,: be that a question ? __eou__,yes __eou__,"thank __eou__ i would imagin so , the site bon...",yes i ve investig that alreadi . it seem you c...,not realli . i use urxvt myself . __eou__,"thank a lot , realli ! __eou__","as someon els suggest , close update-manag , a...",you re welcom .. sinc 12.04 throw dnsmasq into...
2,in my countri it near the 27th __eou__ when wi...,thanx __eou__,"i have no .docx file , so do n't know , whi no...",i ve boot countless distro from usb on my aao ...,but i 'm sure i can work it out __eou__,"the way you put it , that sound like a sever c...",im not familiar with hotspot __eou__,it work fine without set up an ssh tunnel manu...,so it have two be a two-command process ? __eou__,"and becaus you onli have 3 gb of ram , be not ...",it ok but no error ? then how do you know it a...
3,it 's not out __eou__ __eot__ they probabali b...,wait for mani thing to be setup __eou__ final ...,"that 's right , while chat i regrett make a lo...",afaik it 's best to start at 2mb = 2048k __eou__,"for the most part , you should be instal pytho...",do you overwrit your win instal or can you bro...,for some reason the headphon option doe not ch...,well then i do n't know . can anyth boot on th...,well then i do n't know . can anyth boot on th...,"ya , but i guess you could do a git of your en...",noexec be a mount option . you would have to c...
4,be the ext4 driver stabl ? __eou__ __eot__ i b...,you sound like it 's updat to skynet . ; ) __e...,"ok i will tri that , brb __eou__ it complain a...",ouch __eou__,i do system annalysi and it say everyth pass 1...,not to mention way less complex ... you can ha...,"well , you can , accord to that articl , i als...","if not , i think you can pretti much grab ani ...","gpart ? i do n't want do edit partit , just mo...",i ve tri it . not a fan at all __eou__ i have ...,"ah , okay __eou__"


## Completely Random Predictor
This naive predictor randomly picks one of the responses for each of the rows in the test data set. Therefore, we would expect it to have ~10% accuracy with recall@1, ~20% accuracy with recall@2, ~50% accuracy with recall@5 and 100% accuracy with recall@10.

In [4]:
# Completely random prediction model
def random_predictor(context, test):
    return np.random.choice(len(test), 10, replace=False)

In [5]:
# As a sanity check, let's see if the random predictor performs as expected
y_random = [random_predictor(test.Context[x], test.iloc[x,1:].values) for x in range(len(test))]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y_random, k)))

Recall @ 1, 10 total choices: 0.0974101
Recall @ 2, 10 total choices: 0.195032
Recall @ 5, 10 total choices: 0.493869
Recall @ 10, 10 total choices: 1


The random predictor does indeed perform as expected.

## TF-IDF Weighting
Now let's try to use **TF-IDF** weighting to vectorize the contexts and responses and use **cosine similarity** to compute the rank each of the possible responses.

In [5]:
# Get data into right format i.e. list of strings
train_data = np.append(train.Context.values,train.Utterance.values)

In [3]:
# Fit TF-IDF model
tfidf_model = tfidf.TFIDF_Predictor()
tfidf_model.train(train_data)

In [10]:
# Evaluate model performance
y = [tfidf_model.predict(test.Context[x], test.iloc[x,1:].values) for x in range(test.shape[0])]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.495032
Recall @ 2, 10 total choices: 0.596882
Recall @ 5, 10 total choices: 0.766121
Recall @ 10, 10 total choices: 1


## word2vec
Next let's generate word embeddings using word2vec and average the word2vec representations for each word in a message in order to get a feature vector for that message. We'll try to using word mover's distance to compute the rank for each of the possible responses. 

In [13]:
# Split each of the messages into lists of words
train_data_lists = [m.split() for m in train_data]

In [15]:
# Train w2v model
# TODO: Toggle these parameters later
w2v_model = word2vec.Word2Vec(train_data_lists, size=100, min_count=3, iter=3)

In [17]:
# Save model
w2v_model.save('w2v_models/size100_mincount3_iter3')

In [34]:
def w2v_predict(model, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses
    
    Args:
        model: a word2vec model
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    context_split = context.split()
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the averaged word2vec vector of response and context
        response_split = response.split()
        sim = model.wmdistance(context_split, response_split)
        sims.append(sim)
    
    return np.argsort(sims, axis=0)[::-1]

In [35]:
# Evaluate model performance
y = [w2v_predict(w2v_model, test.Context[x], test.iloc[x,1:].values) for x in range(test.shape[0])]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.0620507
Recall @ 2, 10 total choices: 0.128118
Recall @ 5, 10 total choices: 0.323943
Recall @ 10, 10 total choices: 1


**Word mover's distance** doesn't appear to be a good similarity measure here, most likely due to it being highly sensitive to differences between message lengths.

In [36]:
import timeit

start = timeit.default_timer()

w2v_model2 = word2vec.Word2Vec(train_data_lists, size=200, min_count=5, iter=20, workers=4)
# Save model
w2v_model2.save('w2v_models/size200_mincount5_iter20')

stop = timeit.default_timer()

print stop - start 

1899.75511289


## K-Means Clustering and Naive Bayes
Proof of concept for the system that we are implementing. First we cluster the doc2vec vectorizations of each text using k-means. Then we train a naive bayes classifier using the clustering for labels. We generate vectors for new texts using the posterior probabilities for each cluster based on Naive Bayes.

In [4]:
# Load trained doc2vec model
d2v2 = doc2vec.Doc2Vec.load("d2v_models/d2v_size200_iter20")

In [5]:
# Create array of doc2vec vectors
X = []
for v in d2v2.docvecs:
    X.append(v)

X = np.array(X)


In [6]:
# Cluster messages using minibatch k-means
mbkmeans = MiniBatchKMeans(n_clusters=100).fit(X)

/Users/harryxue/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1279: DeprecationWarning: This function is deprecated. Please call randint(0, 1999999 + 1) instead
  0, n_samples - 1, init_size)
/Users/harryxue/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1999999 + 1) instead
  0, n_samples - 1, init_size)
/Users/harryxue/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1999999 + 1) instead
  0, n_samples - 1, init_size)
/Users/harryxue/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 1999999 + 1) instead
  0, n_samples - 1, init_size)
/Users/harryxue/anaconda/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:1328: DeprecationWarning: This function is deprecated. Please call randint(0, 

In [7]:
# Train naive bayes classifier using clustering as labels
from sklearn.naive_bayes import MultinomialNB, GaussianNB
nb = GaussianNB()
nb.fit(X, mbkmeans.labels_)

GaussianNB()

In [26]:
def nb_predict(nb_model, d2v_model, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses
    
    Args:
        nb_model: a trained naive bayes classifier
        d2v_model: a trained doc2vec model
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    # Get vector of probabilities for each of the clusters for the context
    context_vector = nb_model.predict_proba(d2v_model.infer_vector(context.split()).reshape(1, -1))[0]
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the doc2vec vectors of response and context
        response_vector = nb_model.predict_proba(d2v_model.infer_vector(response.split()).reshape(1, -1))[0]
        sim = cosine_similarity(context_vector.reshape(1, -1), response_vector.reshape(1, -1))[0][0]
        sims.append(sim)
        
    print sims
    
    return np.argsort(sims, axis=0)[::-1]

In [27]:
# Evaluate model performance with clustering + naive bayes approach
y = [nb_predict(nb, d2v2, test.Context[x], test.iloc[x,1:].values) for x in range(1000)]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

[1.6463364044178114e-43, 6.7255453983486163e-17, 4.7138373994637225e-22, 5.3480802223449609e-29, 9.6694847758258232e-12, 1.3429310948444991e-12, 2.3746561660523357e-21, 5.6108196078118595e-28, 1.3996129956853273e-27, 1.2113011237533662e-48]
Recall @ 1, 10 total choices: 0
Recall @ 2, 10 total choices: 0
Recall @ 5, 10 total choices: 0
Recall @ 10, 10 total choices: 1


## doc2vec

In [4]:
taggedMessage = namedtuple('TaggedMessage', 'words tags')
documents = []

# Preprocess messages
for i, message in enumerate(train_data):
    # Split into lists of words
    words = message.split()
    tags = [i]
    x = taggedMessage(words, tags)
    documents.append(taggedMessage(words, tags))

In [7]:
# Train doc2vec model
start = timeit.default_timer()
d2v = doc2vec.Doc2Vec(documents, size=100, workers=4, iter=5)
stop = timeit.default_timer()
print stop - start 

1040.17123508


In [8]:
d2v.save("d2v_size100_iter5")

In [9]:
# Train doc2vec model with different stuff
start = timeit.default_timer()
d2v2 = doc2vec.Doc2Vec(documents, size=200, workers=4, iter=20)
stop = timeit.default_timer()
print stop - start 

5021.152601


In [10]:
d2v2.save("d2v_size200_iter20")

In [15]:
def d2v_predict(model, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses
    
    Args:
        model: a doc2vec model
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    context_vector = model.infer_vector(context.split())
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the doc2vec vectors of response and context
        response_vector = model.infer_vector(response.split())
        sim = cosine_similarity(context_vector.reshape(1, -1), response_vector.reshape(1, -1))[0][0]
        sims.append(sim)
    
    return np.argsort(sims, axis=0)[::-1]

In [17]:
# Evaluate model performance with doc2vec model with size 200, 20 iterations
y = [d2v_predict(d2v2, test.Context[x], test.iloc[x,1:].values) for x in range(test.shape[0])]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.333245
Recall @ 2, 10 total choices: 0.477378
Recall @ 5, 10 total choices: 0.737474
Recall @ 10, 10 total choices: 1


Our best doc2vec model performs slightly worse than TF-IDF. This may have to do with the length discrepancy between contexts and response. It may be interesting to see how doc2vec performs with datasets with similar message lengths.

In [18]:
# Evaluate model performance with doc2vec model with size 100, 5 iterations
y = [d2v_predict(d2v, test.Context[x], test.iloc[x,1:].values) for x in range(test.shape[0])]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))

Recall @ 1, 10 total choices: 0.280391
Recall @ 2, 10 total choices: 0.423044
Recall @ 5, 10 total choices: 0.70222
Recall @ 10, 10 total choices: 1


As expected, the doc2vec model which took less training time performs more poorly.

## Export best doc2vec model to tsv
Next let's export our best doc2vec model's vectors to tsv for visualization with Google's embedding projector (http://projector.tensorflow.org/)

In [57]:
# Write a tsv file with one doc2vec per row
with open('tsv/doc2vec_first10000.tsv', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
#     for v in d2v2.docvecs:
#         writer.writerow(v)
    for i in range(10000):
        writer.writerow(d2v2.docvecs[i])

In [58]:
# Write tsv file with metadata (i.e. all the training text, one per line)
with open('tsv/metadata_first10000.tsv', 'w') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    for doc in train_data[:10000]:
        writer.writerow([doc])

## LDA

In [38]:
# Generate corpus for use with LDA
texts = [doc.split() for doc in train_data]

In [40]:
# Create dictionary for corpus
dictionary = gensim.corpora.Dictionary(texts)
# Save dictionary for use later
dictionary.save('dict/ubuntu.dict')

In [41]:
# Create corpus of bag of words
corpus = [dictionary.doc2bow(text) for text in texts]
gensim.corpora.MmCorpus.serialize('/corpora/ubuntu_bow.mm', corpus)  # store to disk, for later use

NameError: name 'corpora' is not defined

In [ ]:
start = timeit.default_timer()
# Train LDA model
lda = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics=100, workers=4)
stop = timeit.default_timer()
print stop - start 

In [4]:
# Load trained LDA model
lda = gensim.models.LdaModel.load("LDA/lda_model1")

In [12]:
lda.print_topics(20)

[(99,
  u'0.135*"__eou__" + 0.076*")" + 0.075*":" + 0.035*"__eot__" + 0.019*"p" + 0.019*"you" + 0.017*"(" + 0.017*"?" + 0.016*"i" + 0.015*";"'),
 (98,
  u'0.174*"__eou__" + 0.100*"!" + 0.081*"thank" + 0.037*"," + 0.033*"__eot__" + 0.030*"i" + 0.026*"." + 0.017*"you" + 0.017*"?" + 0.016*"ok"'),
 (39,
  u'0.049*"the" + 0.048*"__eou__" + 0.040*"," + 0.035*"i" + 0.033*"instal" + 0.032*"boot" + 0.027*"to" + 0.026*"be" + 0.022*"it" + 0.021*"grub"'),
 (92,
  u'0.094*"__eou__" + 0.053*"?" + 0.044*"__eot__" + 0.043*"i" + 0.030*"to" + 0.029*"do" + 0.027*"it" + 0.022*"you" + 0.019*"how" + 0.018*"the"'),
 (60,
  u'0.065*"__eou__" + 0.033*"i" + 0.030*"the" + 0.028*"__eot__" + 0.026*"be" + 0.026*"to" + 0.026*"." + 0.026*"," + 0.023*"you" + 0.021*"?"'),
 (69,
  u'0.087*"__eou__" + 0.041*"__eot__" + 0.035*"," + 0.026*"i" + 0.023*"?" + 0.021*"be" + 0.018*"a" + 0.017*"it" + 0.017*"the" + 0.015*"."'),
 (43,
  u'0.079*"__eou__" + 0.043*"__eot__" + 0.035*"," + 0.027*"i" + 0.026*"be" + 0.024*"to" + 0.024*"?

In [13]:
# load saved dictionary
dictionary = gensim.corpora.dictionary.Dictionary.load("dict/ubuntu.dict")

In [90]:
# Convert gensim LDA topic distribution for a given document to feature vector of length number of topics
def get_lda_feature_vector(topic_dist, num_topics):
    vector = np.zeros(num_topics)
    
    # Fill in values for nonzero topics
    for index, value in topic_dist:
        vector[index] = value
            
    return vector

In [91]:
def lda_predict(model, dictionary, context, responses):
    '''
    Calculates the cosine similarity between the context and each of the possible responses,
    returning a ranked list of responses sorted in decreasing order by cosine similarity
    
    Args:
        model: a lda model
        dictionary: dictionary associated with the lda model
        context: a context that we want to find the response for
        responses: list of candidate responses containing the actual response
        
    Returns:
        List of response indices sorted in descending order by cosine similarity with context
    '''
    # Infer topic distribution and vectorize
    context_vector = get_lda_feature_vector(model[dictionary.doc2bow(context.split())], model.num_topics)
    sims = []
    
    for response in responses:
        # Calculate cosine similarity between the lda feature vectors of response and context
        response_vector = get_lda_feature_vector(model[dictionary.doc2bow(response.split())], model.num_topics)
        sim = cosine_similarity(context_vector.reshape(1, -1), response_vector.reshape(1, -1))[0][0]
        sims.append(sim)
        
    return np.argsort(sims, axis=0)[::-1]

In [ ]:
# Evaluate model performance with lda model
start = timeit.default_timer()
y = [lda_predict(lda, dictionary, test.Context[x], test.iloc[x,1:].values) for x in range(1000)]
for k in [1, 2, 5, 10]:
    print("Recall @ {}, 10 total choices: {:g}".format(k, recall_at_k(y, k)))
stop = timeit.default_timer()

print stop - start